# South African Language Identification Hack 2023

## Installing necessary files

In [ ]:
!pip install parfit
!pip install scikit-plot
!pip install contractions


  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 10.1 MB/s eta 0:00:00


In [ ]:
!pip install wordcloud

Imports for Natural Language processing

In [101]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import re
from sklearn.pipeline import Pipeline
import pickle
import nltk
import string

Library for Data cleaning and preprocessing

In [102]:
from wordcloud import WordCloud, STOPWORDS , ImageColorGenerator
from nltk.tokenize import word_tokenize, TreebankWordTokenizer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils import resample

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB


Loading the data


In [63]:
from google.colab import files
uploaded_files = files.upload()

Saving test_set.csv to test_set (2).csv
Saving train_set.csv to train_set (1).csv


In [14]:
from google.colab import files
uploaded_files = files.upload()

Saving train_set.csv to train_set.csv


In [104]:
train_df = pd.read_csv('train_set.csv')
test_df = pd.read_csv('test_set.csv')

## Exploratory Data Analysis (EDA)

In [105]:
print("Train Dataset:")
print(train_df.head())

Train Dataset:
  lang_id                                               text
0     xho  umgaqo-siseko wenza amalungiselelo kumaziko ax...
1     xho  i-dha iya kuba nobulumko bokubeka umsebenzi na...
2     eng  the province of kwazulu-natal department of tr...
3     nso  o netefatša gore o ba file dilo ka moka tše le...
4     ven  khomishini ya ndinganyiso ya mbeu yo ewa maana...


In [106]:
print("\nTest Dataset:")
print(test_df.head())


Test Dataset:
   index                                               text
0      1  Mmasepala, fa maemo a a kgethegileng a letlele...
1      2  Uzakwaziswa ngokufaneleko nakungafuneka eminye...
2      3         Tshivhumbeo tshi fana na ngano dza vhathu.
3      4  Kube inja nelikati betingevakala kutsi titsini...
4      5                      Winste op buitelandse valuta.


In [107]:
# checking the shape of the dataframe

train_df.shape

(33000, 2)

In [108]:
# Let's look at the information of the dataset

train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33000 entries, 0 to 32999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   lang_id  33000 non-null  object
 1   text     33000 non-null  object
dtypes: object(2)
memory usage: 515.8+ KB


In [109]:
train_df['text']

0        umgaqo-siseko wenza amalungiselelo kumaziko ax...
1        i-dha iya kuba nobulumko bokubeka umsebenzi na...
2        the province of kwazulu-natal department of tr...
3        o netefatša gore o ba file dilo ka moka tše le...
4        khomishini ya ndinganyiso ya mbeu yo ewa maana...
                               ...                        
32995    popo ya dipolateforomo tse ke go tlisa boetele...
32996    modise mosadi na o ntse o sa utlwe hore thaban...
32997    closing date for the submission of completed t...
32998    nawuphina umntu ofunyenwe enetyala phantsi kwa...
32999    mafapha a mang le ona a lokela ho etsa ditlale...
Name: text, Length: 33000, dtype: object

In [110]:
train = train_df
test = test_df

## Data Preprocessing

1. We will be cleaning the data by removing punctuation

In [111]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [112]:
def remove_punctuation(text):
  return ''.join([l for l in text if l not in string.punctuation])

In [113]:
train['text'] = train['text'].apply(remove_punctuation)
train['text'].iloc[:]

0        umgaqosiseko wenza amalungiselelo kumaziko axh...
1        idha iya kuba nobulumko bokubeka umsebenzi nap...
2        the province of kwazulunatal department of tra...
3        o netefatša gore o ba file dilo ka moka tše le...
4        khomishini ya ndinganyiso ya mbeu yo ewa maana...
                               ...                        
32995    popo ya dipolateforomo tse ke go tlisa boetele...
32996    modise mosadi na o ntse o sa utlwe hore thaban...
32997    closing date for the submission of completed t...
32998    nawuphina umntu ofunyenwe enetyala phantsi kwa...
32999    mafapha a mang le ona a lokela ho etsa ditlale...
Name: text, Length: 33000, dtype: object

In [114]:
test['text'] = test['text'].apply(remove_punctuation)
test['text'].iloc[:]

0       Mmasepala fa maemo a a kgethegileng a letlelel...
1       Uzakwaziswa ngokufaneleko nakungafuneka eminye...
2               Tshivhumbeo tshi fana na ngano dza vhathu
3       Kube inja nelikati betingevakala kutsi titsini...
4                            Winste op buitelandse valuta
                              ...                        
5677                      You mark your ballot in private
5678    Ge o ka kgetha ka bowena go se šomiše Mofani k...
5679    E Ka kopo etsa kgetho ya hao ka hloko hobane h...
5680    TB ke bokudi ba PMB mme Morero o tla lefella t...
5681                  Vakatjhela iwebhusayidi yethu kuwww
Name: text, Length: 5682, dtype: object

In [115]:
train['text'] = train['text'].str.lower()

In [116]:
test['text'] = test['text'].str.lower()

In [117]:
test

,index,text
0,1,mmasepala fa maemo a a kgethegileng a letlelel...
1,2,uzakwaziswa ngokufaneleko nakungafuneka eminye...
2,3,tshivhumbeo tshi fana na ngano dza vhathu
3,4,kube inja nelikati betingevakala kutsi titsini...
4,5,winste op buitelandse valuta
...,...,...
5677,5678,you mark your ballot in private
5678,5679,ge o ka kgetha ka bowena go se šomiše mofani k...
5679,5680,e ka kopo etsa kgetho ya hao ka hloko hobane h...
5680,5681,tb ke bokudi ba pmb mme morero o tla lefella t...


2. Tokenise to break down text sequence

In [118]:
tokeniser = TreebankWordTokenizer()
train['tokens'] = train['text'].apply(tokeniser.tokenize)
train['tokens']. iloc[:]

0        [umgaqosiseko, wenza, amalungiselelo, kumaziko...
1        [idha, iya, kuba, nobulumko, bokubeka, umseben...
2        [the, province, of, kwazulunatal, department, ...
3        [o, netefatša, gore, o, ba, file, dilo, ka, mo...
4        [khomishini, ya, ndinganyiso, ya, mbeu, yo, ew...
                               ...                        
32995    [popo, ya, dipolateforomo, tse, ke, go, tlisa,...
32996    [modise, mosadi, na, o, ntse, o, sa, utlwe, ho...
32997    [closing, date, for, the, submission, of, comp...
32998    [nawuphina, umntu, ofunyenwe, enetyala, phants...
32999    [mafapha, a, mang, le, ona, a, lokela, ho, ets...
Name: tokens, Length: 33000, dtype: object

In [119]:
tokeniser = TreebankWordTokenizer()
test['tokens'] = test['text'].apply(tokeniser.tokenize)
test['tokens']. iloc[:]

0       [mmasepala, fa, maemo, a, a, kgethegileng, a, ...
1       [uzakwaziswa, ngokufaneleko, nakungafuneka, em...
2       [tshivhumbeo, tshi, fana, na, ngano, dza, vhathu]
3       [kube, inja, nelikati, betingevakala, kutsi, t...
4                       [winste, op, buitelandse, valuta]
                              ...                        
5677               [you, mark, your, ballot, in, private]
5678    [ge, o, ka, kgetha, ka, bowena, go, se, šomiše...
5679    [e, ka, kopo, etsa, kgetho, ya, hao, ka, hloko...
5680    [tb, ke, bokudi, ba, pmb, mme, morero, o, tla,...
5681             [vakatjhela, iwebhusayidi, yethu, kuwww]
Name: tokens, Length: 5682, dtype: object

In [120]:
def lemmatize_text(text):
    wordnet_lemmatizer = WordNetLemmatizer()
    lemmatized_words = []
    for word in word_tokenize(text):
        lemmatized_words.append(wordnet_lemmatizer.lemmatize(word))
    return " ".join(lemmatized_words)


In [121]:
#lemmatized_text = lemmatize_text(words)
#print(lemmatized_text)

## Feature Engineering

In [122]:
# splitting x and y variables
X = train['text']
y = train['lang_id']

## Train Test Split

In [123]:
# Splitting the dataset into 80/20

X_train , X_test , y_train , y_test = train_test_split(X , y, test_size =0.20, random_state=42)

## Model

In [124]:
def preprocess_data(train, test):
    # Initializing the TF-IDF vectorizer
    vectorizer = TfidfVectorizer()

    # Fitting the vectorizer on the training data
    vectorizer.fit(train['text'])

    # Transforming the training and test data using the fitted vectorizer
    train_features = vectorizer.transform(train['text'])
    test_features = vectorizer.transform(test['text'])

    return train_features, test_features, vectorizer


## Preprocessed Data

In [125]:
train_features, test_features, vectorizer = preprocess_data(train, test)


## Training and Evaluation

Logistic Regression

In [126]:
X_train, X_test, y_train, y_test = train_test_split(train_features, train['lang_id'], test_size=0.2, random_state=42)
lr_model = LogisticRegression()
lr_model.fit(X_train, y_train)
lr_preds = lr_model.predict(X_test)
lr_f1 = f1_score(y_test, lr_preds, average='weighted')

print("Logistic Regression F1 Score:", lr_f1)


Logistic Regression F1 Score: 0.9943976987335509


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


K Nearest Neighbours (KNN)

In [127]:
knn_model = KNeighborsClassifier()
knn_model.fit(X_train, y_train)
knn_preds = knn_model.predict(X_test)
knn_f1 = f1_score(y_test, knn_preds, average='weighted')

print("KNN F1 Score:", knn_f1)

KNN F1 Score: 0.9620927040666665


Support Vector Machine

In [128]:
svm = SVC()
svm.fit(X_train, y_train)
svm_predictions = svm.predict(X_test)
svm_f1 = f1_score(y_test, svm_predictions, average='weighted')
print("SVM F1 Score:", svm_f1)

SVM F1 Score: 0.9942548884498323


Naive Bayes

In [129]:
nb = MultinomialNB()
nb.fit(X_train, y_train)
nb_predictions = nb.predict(X_test)
nb_f1 = f1_score(y_test, nb_predictions, average='weighted')
print("Naive Bayes F1 Score:", nb_f1)

Naive Bayes F1 Score: 0.9984848537376715


Generate results on test set

In [138]:
# Converting the test data into TF-IDF vectors
X_test = vectorizer.transform(test['text'])

# Generating predictions on the best performing model
test_predictions = nb.predict(X_test)

Creating a CSV submission

In [139]:
# Creating a submission dataframe with 'index' and 'lang_id' columns
submission_df = pd.DataFrame({'index': test['index'], 'lang_id': test_predictions})

submission_df.to_csv('Final2_Sub.csv', index=False)